In [1]:
from llmfact import LayerOutputExtractor, FBNFeatureExtractor, GroupFBNFeatureExtractor, FBNExtractor, LLMFC
from llmfact.extractor import MutiLayerAnalysis, MutiLayerAnalysis2
from llmfact.extractor import SingleLayerAnalysis
from llmfact.mask import MaskedGPT2ForSequenceClassification, MaskedGPT2AmplifiedForSequenceClassification, MaskedGPT2LMModel, MaskedModel
from transformers import GPT2Model, GPT2Config, GPT2LMHeadModel, GPT2ForSequenceClassification, Trainer, TrainingArguments
from transformers import GPT2Tokenizer
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForCausalLM, AutoModelForQuestionAnswering
from datasets import load_dataset, get_dataset_config_names, get_dataset_split_names
from torch.utils.data import DataLoader
# from rouge_score import rouge_scorer
from evaluate import load

from llmfact.utils import IoU, correlation_activation, thresholding, write_layer_txt, evaluate_iou
from llmfact.stat import  StatICA, StatDictionaryLearning
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import FastICA
import seaborn as sns
import pandas as pd
from tqdm.auto import tqdm
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7,8,9'
os.environ["TOKENIZERS_PARALLELISM"]  = "true"

In [2]:
model_name = "lmsys/vicuna-7b-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
include_layers = []
for name, _ in model.named_modules():
    if "mlp.act" in name or "mlp.up" in name:
        include_layers.append(name)
include_layers

['model.layers.0.mlp.up_proj',
 'model.layers.0.mlp.act_fn',
 'model.layers.1.mlp.up_proj',
 'model.layers.1.mlp.act_fn',
 'model.layers.2.mlp.up_proj',
 'model.layers.2.mlp.act_fn',
 'model.layers.3.mlp.up_proj',
 'model.layers.3.mlp.act_fn',
 'model.layers.4.mlp.up_proj',
 'model.layers.4.mlp.act_fn',
 'model.layers.5.mlp.up_proj',
 'model.layers.5.mlp.act_fn',
 'model.layers.6.mlp.up_proj',
 'model.layers.6.mlp.act_fn',
 'model.layers.7.mlp.up_proj',
 'model.layers.7.mlp.act_fn',
 'model.layers.8.mlp.up_proj',
 'model.layers.8.mlp.act_fn',
 'model.layers.9.mlp.up_proj',
 'model.layers.9.mlp.act_fn',
 'model.layers.10.mlp.up_proj',
 'model.layers.10.mlp.act_fn',
 'model.layers.11.mlp.up_proj',
 'model.layers.11.mlp.act_fn',
 'model.layers.12.mlp.up_proj',
 'model.layers.12.mlp.act_fn',
 'model.layers.13.mlp.up_proj',
 'model.layers.13.mlp.act_fn',
 'model.layers.14.mlp.up_proj',
 'model.layers.14.mlp.act_fn',
 'model.layers.15.mlp.up_proj',
 'model.layers.15.mlp.act_fn',
 'model.laye

In [4]:
wiki_dataset = load_dataset("Self-GRIT/wikitext-2-raw-v1-preprocessed", split='train')
print(wiki_dataset)

Dataset({
    features: ['text'],
    num_rows: 15313
})


In [5]:
ica_dataset = []
sample_num = 40
ica_num = 1
for i in range(sample_num * ica_num):
    ica_dataset.append(wiki_dataset['text'][i])

In [9]:
from llmfact.decomposition.canica import CanICA

def z_score_signals(signals):
	if not isinstance(signals, torch.Tensor):
		raise TypeError("Input signals must be a PyTorch tensor.")

	mean = torch.mean(signals, dim=0)
	std = torch.std(signals, dim=0)

	signals = signals - mean

	eps = torch.finfo(signals.dtype).eps  # 获取当前数据类型的最小正数
	std = torch.where(std < eps, torch.tensor(1.0, dtype=signals.dtype, device=signals.device), std)

	# 归一化
	signals /= std

	return signals
    
class SingleLayerAnalysis(LayerOutputExtractor):
    def __init__(self, model, include_layers=["h.0.attn.c_attn"], test=False, device='cpu'):
        super().__init__(model, include_layers=include_layers, test=test, device=device)
        self.include_layers = include_layers
        self.mixing_ = None
        self.origin_mixing_ = None
        self.normal_mixing_ = None

    def fit(self, inputs, n_components=10, alpha=1.96, random_state=666,
            preprocessing=False, total_layer_num=None, method="fastica",
            max_iter=200, n_iter=5, norm=True):
        if total_layer_num:
            total_layer_num = total_layer_num
        else:
            total_layer_num = len(self.include_layers)

        if type(inputs) == list:
            total_layer_outputs = []
            for inp in inputs:
                layer_outputs = self.extract_layer_outputs(inp)
                total_layer_outputs.append(layer_outputs)
            layer_outputs = torch.cat(total_layer_outputs, dim=0)
        else:
            layer_outputs = self.extract_layer_outputs(inputs)

        if preprocessing:
            layer_outputs = z_score_signals(layer_outputs)
            layer_outputs = layer_outputs.to(torch.float64)

        token_num = layer_outputs.shape[0]

        layer_outputs = layer_outputs.reshape(token_num, total_layer_num, -1)

        # mixing_list = []
        # origin_mixing_list = []
        normal_mixing_list = []
        for i in trange(total_layer_num):
            if method == "fastica":
                ica = FastICA(n_components=n_components,
                              random_state=random_state,
                              max_iter=max_iter)
                ica.fit(layer_outputs[:, i, :])
                mixing = torch.tensor(ica.mixing_.T)
                # origin_mixing_list.append(mixing)

                mean = torch.mean(mixing, dim=1, keepdim=True)
                std = torch.std(mixing, dim=1, keepdim=True)

                normalized_matrix = (mixing - mean) / std

                normal_mixing_list.append(normalized_matrix)

            else:
                ica = CanICA(n_components=n_components, random_state=random_state, device=self.device)
                ica.fit(layer_outputs[:, i, :], max_iter=max_iter)

                normal_mixing_list.append(ica.normal_mixing_)
                
        self.normal_mixing_ = torch.cat(normal_mixing_list, dim=1)

In [10]:
def MutiICA(model, include_layers, dataset, n_components=256, window_size=3, preprocessing=True, max_iter=500, n_iter=5, norm=True):
    extractor = MutiLayerAnalysis2(model, include_layers=include_layers, device=model.device)
    inputs_list = [tokenizer(inputs, return_tensors="pt", max_length=1024, truncation=True) for inputs in dataset]
    extractor.fit(inputs=inputs_list, n_components=n_components, 
                  window_size=window_size, random_state=666,
                  preprocessing=preprocessing, total_layer_num=32, method="canica", max_iter=max_iter, norm=norm, n_iter=n_iter)
    return extractor
    

def save_fbn(data, save_dir, data_type, model_name, n_components, alpha):
    if isinstance(data, list) or isinstance(data, dict):
        save_path = save_dir + data_type + "_" + model_name + "_" + str(n_components) + "_" + str(alpha) + ".pth"
        print(f"save at {save_path}")
        torch.save(data, save_path)
    else:
        save_path = save_dir + data_type + "_" + model_name + "_" + str(n_components) + "_" + str(alpha) + ".pth"
        print(f"save at {save_path}")
        np.save(save_path, data)

def SingleICA(model, include_layers, dataset, n_components=256, preprocessing=True, max_iter=500, norm=True, n_iter=5):
    extractor = SingleLayerAnalysis(model, include_layers=include_layers, device=model.device)
    inputs_list = [tokenizer(inputs, return_tensors="pt", max_length=1024, truncation=True) for inputs in dataset]
    extractor.fit(inputs=inputs_list, n_components=n_components, 
                  random_state=327, preprocessing=True,
                  total_layer_num=32, method="canica", max_iter=max_iter, norm=norm, n_iter=n_iter)
    return extractor

In [11]:
len(ica_dataset)

40

In [8]:
%%time
from tqdm import trange
import warnings
warnings.filterwarnings("ignore")

model_name = "vicuna-7b-v1.5-muti-layer-wise"
n_components = 128
preprocessing = True
max_iter = 300
norm = False
n_iter = 4

normal_list = []
for i in trange(ica_num):
    try:
        extractor = SingleICA(model, include_layers, ica_dataset[sample_num*i:sample_num*(i+1)],
                              n_components=n_components, preprocessing=True,
                              max_iter=max_iter, norm=norm, n_iter=n_iter)
        normal = extractor.normal_mixing_
        normal_list.append(normal)
    except:
        print(f"第{i}次算ICA出现问题，出现问题的样本在[{sample_num*i}:{sample_num*(i+1)}]")
normal_components = np.concatenate(normal_list, axis=0)
save_fbn(normal_components, "./data/FBN/", f"text{len(ica_dataset)}-mlp.act-CanICA-SingleICA-max_iter-{max_iter}", model_name, n_components, f"normal_mixing_std_{preprocessing}")
normal_components.shape

  0%|          | 0/32 [00:00<?, ?it/s]/data4/lyh/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(

 97%|█████████▋| 31/32 [26:27<00:54, 54.33s/it]/data4/lyh/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/data4/lyh/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/data4/lyh/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.

第11次算ICA出现问题，出现问题的样本在[440:480]


 72%|███████▏  | 23/32 [18:41<07:44, 51.65s/it]/data4/lyh/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(

 78%|███████▊  | 25/32 [20:57<06:54, 59.25s/it]/data4/lyh/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(

 97%|█████████▋| 31/32 [27:22<01:01, 61.01s/it]/data4/lyh/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/data4/lyh/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing t

第54次算ICA出现问题，出现问题的样本在[2160:2200]


 97%|█████████▋| 31/32 [26:45<00:59, 59.01s/it]/data4/lyh/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/data4/lyh/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/data4/lyh/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/data4/lyh/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/data4/lyh/anaconda3/envs/torch/l

save at ./data/FBN/text3200-mlp.act-CanICA-SingleICA-max_iter-300_vicuna-7b-v1.5-muti-layer-wise_128_normal_mixing_std_True.npy
CPU times: user 12d 16h 23min 42s, sys: 2h 50min 41s, total: 12d 19h 14min 23s
Wall time: 1d 19h 29min 37s


(9984, 704512)

In [10]:
def cut_par_num(neuron_num_list):
    total_par = 6738415616
    print("total parameters:", total_par)
    
    total_mlp = 32 * (4096 * 11008 * 3)
    print("total mlp parameters:", total_mlp)

    total_cut = 0
    for i in neuron_num_list:
        cut_num = 4096 * 11008 * 3 - (i * 4096 * 3)
        total_cut += cut_num
    print("total cut parameters num:", total_cut)

    print(f"total cut mlp parameters: {total_cut / total_mlp:.4f}")
    print(f"total cut parameters: {total_cut / total_par:.4f}")
    print(f"parameters after cut: {total_par - total_cut:.4f}")

In [13]:
any_mask = np.abs(normal_components) > 6.03
any_mask = np.any(any_mask, axis=0).reshape(1, -1)
print(any_mask.sum())

mask = any_mask.reshape(32, 2, -1)
mask_matrix = np.ones((32, 11008))
for i in range(3, mask.shape[0] - 2):
    mask_matrix[i] = np.any(mask[i], axis=0)
print(mask_matrix.sum())
print(mask_matrix.sum(axis=1))
cut_par_num(mask_matrix.sum(axis=1))
# mask_matrix = np.repeat(mask_matrix, 2, axis=0)
mask_matrix.shape

276714
243438.0
[11008. 11008. 11008.  4776.  5473.  6001.  6571.  7220.  7181.  7296.
  7414.  7467.  7488.  7553.  7594.  7533.  7887.  7633.  7596.  7212.
  7295.  7189.  7310.  7064.  7157.  6829.  6560.  6564.  6256.  6279.
 11008. 11008.]
total parameters: 6738415616
total mlp parameters: 4328521728
total cut parameters num: 1337155584.0
total cut mlp parameters: 0.3089
total cut parameters: 0.1984
parameters after cut: 5401260032.0000


(32, 11008)

In [14]:
from llmfact.pruner.pruner import PrunedLlamaModel
pruner = PrunedLlamaModel(model, mask_matrix)
model = pruner.fit()

total parameters before pruned: 6738415616
total parameters after pruned: 5401260032
total cut num: 1337155584
pruned rate: 0.1984


In [2]:
(6738415616 - 5444132864) / 6738415616

0.1920752333718918

In [15]:
from lm_eval import evaluator
import lm_eval
wrapper_model = lm_eval.models.huggingface.HFLM(pretrained=model, trust_remote_code=True)

`pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration


In [16]:
results = evaluator.simple_evaluate( 
    model=wrapper_model,
    model_args="lmsys/vicuna-7b-v1.5",
    tasks=["wikitext"],
    num_fewshot=0,
    task_manager=lm_eval.tasks.TaskManager(),
    batch_size=1)
results['results']

[Task: wikitext] metric word_perplexity is defined, but aggregation is not. using default aggregation=weighted_perplexity
[Task: wikitext] metric word_perplexity is defined, but higher_is_better is not. using default higher_is_better=False
[Task: wikitext] metric byte_perplexity is defined, but aggregation is not. using default aggregation=weighted_perplexity
[Task: wikitext] metric byte_perplexity is defined, but higher_is_better is not. using default higher_is_better=False
[Task: wikitext] metric bits_per_byte is defined, but aggregation is not. using default aggregation=bits_per_byte
[Task: wikitext] metric bits_per_byte is defined, but higher_is_better is not. using default higher_is_better=False
Overwriting default num_fewshot of wikitext from None to 0
Running loglikelihood requests: 100%|██████████| 1/1 [00:06<00:00,  6.07s/it]


{'wikitext': {'alias': 'wikitext',
  'word_perplexity,none': 16.724930608476676,
  'word_perplexity_stderr,none': 'N/A',
  'byte_perplexity,none': 1.6934611737796776,
  'byte_perplexity_stderr,none': 'N/A',
  'bits_per_byte,none': 0.7599749103409951,
  'bits_per_byte_stderr,none': 'N/A'}}

In [17]:
results = evaluator.simple_evaluate( 
    model=wrapper_model,
    tasks=["piqa", "hellaswag", "winogrande", "openbookqa", "arc_easy", "arc_challenge"],
    num_fewshot=0,
    task_manager=lm_eval.tasks.TaskManager(),
    batch_size=1)
results['results']

Overwriting default num_fewshot of arc_challenge from None to 0
Overwriting default num_fewshot of arc_easy from None to 0
Overwriting default num_fewshot of openbookqa from None to 0
Overwriting default num_fewshot of winogrande from None to 0
Overwriting default num_fewshot of hellaswag from None to 0
Overwriting default num_fewshot of piqa from None to 0
Running loglikelihood requests: 100%|██████████| 62566/62566 [3:51:22<00:00,  4.51it/s]  


{'arc_challenge': {'alias': 'arc_challenge',
  'acc,none': 0.3779863481228669,
  'acc_stderr,none': 0.014169664520303164,
  'acc_norm,none': 0.3993174061433447,
  'acc_norm_stderr,none': 0.014312094557946681},
 'arc_easy': {'alias': 'arc_easy',
  'acc,none': 0.6822390572390572,
  'acc_stderr,none': 0.009554033064443069,
  'acc_norm,none': 0.6397306397306397,
  'acc_norm_stderr,none': 0.00985100258473252},
 'hellaswag': {'alias': 'hellaswag',
  'acc,none': 0.49352718581955785,
  'acc_stderr,none': 0.004989363276955276,
  'acc_norm,none': 0.657239593706433,
  'acc_norm_stderr,none': 0.0047366216988608345},
 'openbookqa': {'alias': 'openbookqa',
  'acc,none': 0.3,
  'acc_stderr,none': 0.020514426225627987,
  'acc_norm,none': 0.384,
  'acc_norm_stderr,none': 0.021772369465547118},
 'piqa': {'alias': 'piqa',
  'acc,none': 0.7257889009793254,
  'acc_stderr,none': 0.010408618664933516,
  'acc_norm,none': 0.736126224156692,
  'acc_norm_stderr,none': 0.010282996367695665},
 'winogrande': {'alia